# Puntos de donación de sangre de la Comunidad de Madrid

In [ ]:
import pandas as pd

from tqdm.auto import tqdm

from utils.db import format_column, from_db, gdf_from_df, run_query, to_db
from utils.geocode import google_geocode_address
from utils.selenium import (
    get_uc_driver, get_lista_centros, extraer_detalles_punto_fijo,
    extraer_detalles_centro, URL_PUNTOS_FIJOS, URL_PUNTOS_MOVILES
)

In [ ]:
def get_full_address(row, address_field):
    return ', '.join([row[address_field], row.municipio, 'Community of Madrid', 'Spain'])


def get_gmaps_url(row):
    return f'https://www.google.com/maps?q={row.latitude}+{row.longitude}'


with open('../sql/table_to_geojson.sql') as fp:
    TABLE_TO_GEOJSON_QUERY = fp.read()

## Puntos fijos

Los hospitales son fáciles de scrapear pero hay otros tres puntos que aparecen desordenados en [la web](https://www.comunidad.madrid/servicios/salud/donacion-sangre#puntos-donacion).

Los metemos a mano

In [ ]:
puntos_fijos_no_hospitales = [
    {
        'nombre': 'Centro de Transfusión de la Comunidad de Madrid',
        'id_del_centro': None,
        'horario_de_donaciones': 'De 8:30 a 21 horas, de lunes a sábado (excepto festivos).',
        'ubicacion_de_las_salas_de_donacion': None,
        'observaciones': 'Cuenta con párking para donantes. Medios de transporte: Autobuses: 71, 130, 8. Metro: Valdebernardo (Línea 9)',
        'informacion_al_donante': 'Tlfno: 900 303 530',
        'direccion_postal': 'esquina con Avenida de las Comunidades, Av. de la Democracia, s/n',
        'municipio': 'Madrid',
        'codigo_postal': '28032',
        'informacion_general': '900 303 530',
        'latitude': 40.396334,
        'longitude': -3.615422,
    },
    {
        'nombre': 'Unidad móvil de la Puerta del Sol',
        'id_del_centro': None,
        'horario_de_donaciones': 'De lunes a viernes de 11 a 21 h. Sábados, domingos y festivos de 10 a 21h.',
        'ubicacion_de_las_salas_de_donacion': None,
        'observaciones': None,
        'informacion_al_donante': None,
        'direccion_postal': 'Prta. del Sol, s/n',
        'municipio': 'Madrid',
        'codigo_postal': '28013',
        'informacion_general': None,
        'latitude': 40.416806,
        'longitude': -3.703911,
    },
    {
        'nombre': 'Unidad de Extracción de Sangre de Cruz Roja',
        'id_del_centro': None,
        'horario_de_donaciones': 'De lunes a viernes. De 9 a 20:30 h. (excepto festivos).',
        'ubicacion_de_las_salas_de_donacion': None,
        'observaciones': None,
        'informacion_al_donante': 'Tlfno: 900 303 530',
        'direccion_postal': 'c/ Juan Montalvo, nº 3.',
        'municipio': 'Madrid',
        'codigo_postal': '28040',
        'informacion_general': None,
        'latitude': 40.447684,
        'longitude': -3.709592,
    }
]

Usamos Selenium ya que con requests tenemos que mandar unos tokens codificados en base64 que maemía

In [ ]:
driver = get_uc_driver(False)

puntos_fijos = [extraer_detalles_punto_fijo(pf) for pf in get_lista_centros(driver, URL_PUNTOS_FIJOS)]

centros = []
for pf in tqdm(puntos_fijos):
    centros.append(extraer_detalles_centro(driver, pf['name'], pf['url']))
    
df_fijos = pd.DataFrame(centros)
df_fijos_keys = pd.DataFrame({'original': df_fijos.columns, 'db': df_fijos.columns.map(format_column)})
df_fijos.columns = df_fijos.columns.map(format_column)

# Añadir puntos fijos a mano
df_fijos = pd.concat([pd.DataFrame(puntos_fijos_no_hospitales), df_fijos], ignore_index=True)
df_fijos['url'] = df_fijos.apply(get_gmaps_url, axis=1)
gdf_fijos = gdf_from_df(df_fijos)

Tenemos que eliminar y recrear la vista de `geojson` dependiente de la tabla de puntos fijos

In [ ]:
run_query('DROP VIEW IF EXISTS puntos_fijos_geojson')

to_db(gdf_fijos, 'puntos_fijos')
to_db(df_fijos_keys, 'puntos_fijos_keys')


run_query(TABLE_TO_GEOJSON_QUERY.format(t='puntos_fijos'))

## Puntos móviles

Muy similar a los puntos fijos, sólo que no tienen web individual y pillamos todos los atributos de una lista

In [ ]:
classes = ['municipio', 'ubicacion', 'direccion', 'fecha', 'horario']

lista_centros = get_lista_centros(driver, URL_PUNTOS_MOVILES)
centros_props = []
for elem in lista_centros:
    elem_props = {}
    for c in classes:
        elem_props[c] = elem.find_element_by_class_name(c).text.strip()
    centros_props.append(elem_props)

df_moviles = pd.DataFrame(centros_props)
df_moviles['nombre'] = 'Equipo móvil en ' + df_moviles['ubicacion']
# df_moviles = df_moviles.drop(columns=['ubicacion'])

db_fields = ['nombre', 'municipio', 'direccion', 'fecha', 'horario']
nice_fields = ['Nombre', 'Municipio', 'Dirección', 'Fecha', 'Horario']
df_moviles_keys = pd.DataFrame({'original': nice_fields, 'db': db_fields})

df_moviles.columns = df_moviles.columns.map(format_column)
for text in ['Equipo móvil en ', 'E Móvil en ', 'Equipo Móvil detrás ']:
    df_moviles['direccion'] = df_moviles['direccion'].map(lambda d: d.replace(text, ''))

Geocode and upload. We geocode both the `ubicacion` and `direccion` and keep their scores
based on the `location_type` (ROOFTOP, etc).

Before geocoding, try to use the cache

In [ ]:
try:
    geocoding_cache = from_db('geocoding_cache')
except Exception:
    geocoding_cache = pd.DataFrame(columns=['address', 'longitude', 'latitude', 'location_type', 'score'])

In [ ]:
new_geocoding_list = []
for address_field in ['ubicacion', 'direccion']:
    full_addresses = df_moviles.apply(get_full_address, axis=1, address_field=address_field)
    for full_address in tqdm(full_addresses.drop_duplicates(), desc=address_field):
        # Try to find address in cache
        gc_row = geocoding_cache.loc[geocoding_cache.address == full_address]
        if not gc_row.empty:
            continue

        # Address not found. Must geocode
        print(f'"{full_address}" not found in cache. Geocoding...')
        lng, lat, location_type, score = google_geocode_address(full_address)
        new_geocoding_list.append({
            'address': full_address,
            'longitude': lng,
            'latitude': lat,
            'location_type': location_type,
            'score': score,
        })

new_geocoding_df = pd.DataFrame(new_geocoding_list)
geocoding_cache = pd.concat([geocoding_cache, new_geocoding_df])

geocoding_cache = geocoding_cache.drop_duplicates()

to_db(geocoding_cache, 'geocoding_cache')

In [ ]:
# Geocoding candidates based on "ubicacion"
df_ubicacion = df_moviles[['ubicacion', 'municipio']].copy()
df_ubicacion['address'] = df_ubicacion.apply(get_full_address, axis=1, address_field='ubicacion')
df_ubicacion = pd.merge(df_ubicacion, geocoding_cache, on='address')

# Geocoding candidates based on "direccion"
df_direccion = df_moviles[['direccion', 'municipio']].copy()
df_direccion['address'] = df_direccion.apply(get_full_address, axis=1, address_field='direccion')
df_direccion = pd.merge(df_direccion, geocoding_cache, on='address')

assert len(df_moviles) == len(df_ubicacion) == len(df_direccion)

Get the coordinates from the best candidate address (`direccion` or `ubicacion`)

In [ ]:
df_moviles['latitude'] = None
df_moviles['longitude'] = None
for idx, row in df_moviles.iterrows():
    score_ubicacion = df_ubicacion.loc[idx, 'score']
    score_direccion = df_direccion.loc[idx, 'score']
    if score_direccion > score_ubicacion:
        lng = df_direccion.loc[idx, 'longitude']
        lat = df_direccion.loc[idx, 'latitude']
    else:
        lng = df_ubicacion.loc[idx, 'longitude']
        lat = df_ubicacion.loc[idx, 'latitude']
        
    df_moviles.loc[idx, 'longitude'] = lng
    df_moviles.loc[idx, 'latitude'] = lat

In [ ]:
df_moviles['url'] = df_moviles.apply(get_gmaps_url, axis=1)
df_moviles = df_moviles.drop(columns=['ubicacion'])

In [ ]:
gdf_moviles = gdf_from_df(df_moviles)

In [ ]:
run_query('DROP VIEW IF EXISTS puntos_moviles_geojson')

to_db(gdf_moviles, 'puntos_moviles')
to_db(df_moviles_keys, 'puntos_moviles_keys')


run_query(TABLE_TO_GEOJSON_QUERY.format(t='puntos_moviles'))